Para a parte de Machine Learning, vamos testar alguns algoritmos diferentes e experimentar trabalhar com _feature engineering_.

Vamos começar importando os dados e as bibliotecas necessárias:

Importando bibliotecas

In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

pd.set_option('display.max_columns', None)

df = pd.read_csv('./dataset/cars_train.csv', sep='\t', encoding='utf_16')

df['ano_modelo'] = df.ano_modelo.astype(int)
df['num_fotos'] = df.num_fotos.fillna(0).astype(int)
df['dono_aceita_troca'] = df.dono_aceita_troca.notnull().astype(int)
df['veiculo_único_dono'] = df.veiculo_único_dono.notnull().astype(int)
df['revisoes_concessionaria'] = df.revisoes_concessionaria.notnull().astype(int)
df['ipva_pago'] = df.ipva_pago.notnull().astype(int)
df['veiculo_licenciado'] = df.veiculo_licenciado.notnull().astype(int)
df['garantia_de_fábrica'] = df.garantia_de_fábrica.notnull().astype(int)
df['revisoes_dentro_agenda'] = df.revisoes_dentro_agenda.notnull().astype(int)
df['veiculo_alienado'] = df.veiculo_alienado.notnull().astype(int)
df['entrega_delivery'] = df.entrega_delivery.astype(int)
df['troca'] = df.troca.astype(int)
df['elegivel_revisao'] = df.elegivel_revisao.astype(int)
df['blindado'] = df.blindado.map({'N': 0, 'S': 1})
df['tipo_vendedor'] = df.tipo_vendedor.map({'PJ': 1, 'PF': 0})
df.rename(columns={'tipo_vendedor':'vendedor_PJ'}, inplace=True)

O primeiro modelo que testaremos será o de regressão linear. Isso porque esse algoritmo é um dos mais simples e tem uma boa interpretabilidade dos resultados.

Porém, devemos nos atentar com a quantidade de variáveis categóricas. A princípio iremos deixar de fora algumas delas:

In [61]:
X = df.select_dtypes(np.number).drop(columns=['preco'])
y = df.preco

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=32)

kfold = KFold(n_splits=5, random_state=31, shuffle=True)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

# for i, (train_index, test_index) in enumerate(kfold.split(X, y)):

#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
#     print(f'[{i+1}] - KFold')
#     print(f'X_train {X_train.shape}')
#     print(f'X_test {X_test.shape}')
#     print(f'y_train {y_train.shape}')
#     print(f'y_test {y_test.shape}')

#     pipeline.fit(X_train, y_train)

#     y_pred = pipeline.predict(X_test)

#     coefs_ = pd.DataFrame({'colunas': X_train.columns, 'coeficientes': pipeline.named_steps['regressor'].coef_})

    # display(coefs_)

    # print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
    # # The coefficient of determination: 1 is perfect prediction
    # print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))
    # print('-'*64)

scores = cross_val_score(pipeline, X, y, cv=kfold, scoring='r2')
print(scores)

[0.2648555  0.23138922 0.21854396 0.21688448 0.22949617]


In [98]:
print(df.columns)
pd.get_dummies(df, columns=['tipo', 'cambio', 'cor'], drop_first=True, ).head()

Index(['id', 'num_fotos', 'marca', 'modelo', 'versao', 'ano_de_fabricacao',
       'ano_modelo', 'hodometro', 'cambio', 'num_portas', 'tipo', 'blindado',
       'cor', 'vendedor_PJ', 'cidade_vendedor', 'estado_vendedor',
       'anunciante', 'entrega_delivery', 'troca', 'elegivel_revisao',
       'dono_aceita_troca', 'veiculo_único_dono', 'revisoes_concessionaria',
       'ipva_pago', 'veiculo_licenciado', 'garantia_de_fábrica',
       'revisoes_dentro_agenda', 'veiculo_alienado', 'preco'],
      dtype='object')


,id,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,num_portas,blindado,vendedor_PJ,cidade_vendedor,estado_vendedor,anunciante,entrega_delivery,troca,elegivel_revisao,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado,preco,tipo_Hatchback,tipo_Minivan,tipo_Perua/SW,tipo_Picape,tipo_Sedã,tipo_Utilitário esportivo,cambio_Automatizada DCT,cambio_Automática,cambio_Automática Sequencial,cambio_CVT,cambio_Manual,cambio_Semi-automática,cor_Cinza,cor_Dourado,cor_Prata,cor_Preto,cor_Verde,cor_Vermelho
0,300716223898539419613863097469899222392,8,NISSAN,KICKS,1.6 16V FLEXSTART SL 4P XTRONIC,2017,2017,67772.0,4,0,0,Rio de Janeiro,São Paulo (SP),Pessoa Física,0,0,0,0,0,1,1,1,0,0,0,74732.590084,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,279639842134129588306469566150288644214,8,JEEP,COMPASS,2.0 16V FLEX LIMITED AUTOMÁTICO,2017,2017,62979.0,4,0,0,Belo Horizonte,Minas Gerais (MG),Pessoa Física,0,0,0,1,0,0,1,0,0,0,0,81965.332634,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
2,56414460810621048900295678236538171981,16,KIA,SORENTO,2.4 16V GASOLINA EX 7L AWD AUTOMÁTICO,2018,2019,44070.0,4,0,1,Santos,São Paulo (SP),Loja,1,0,0,1,0,0,0,0,0,0,0,162824.814472,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
3,56862509826849933428086372390159405545,14,VOLKSWAGEN,AMAROK,2.0 HIGHLINE 4X4 CD 16V TURBO INTERCOOLER DIES...,2013,2015,85357.0,4,0,1,Sorocaba,São Paulo (SP),Loja,1,1,0,1,0,0,1,1,0,0,0,123681.358857,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,338980975753200343894519909855598027197,8,SSANGYONG,KORANDO,2.0 GLS 4X4 16V TURBO DIESEL 4P AUTOMÁTICO,2013,2015,71491.0,4,0,0,Rio de Janeiro,Rio de Janeiro (RJ),Pessoa Física,0,0,0,0,0,1,0,0,1,1,0,82419.763891,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [120]:
df1 = pd.get_dummies(df, columns=['tipo', 'cambio', 'cor', 'anunciante'], drop_first=True)

X = df1.select_dtypes(np.number).drop(columns=['preco'])
y = df1.preco

kfold = KFold(n_splits=5, random_state=31, shuffle=True)

pipeline = Pipeline([
    # ('encoder', OneHotEncoder(categories='auto', drop='first')),
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])


scores = cross_val_score(pipeline, X, y, cv=kfold, scoring='r2')
print(scores)

[0.45678729 0.40996232 0.42214345 0.42260468 0.42376281]


In [117]:
set(pd.get_dummies(df, columns=['tipo', 'cambio', 'cor', 'anunciante'])) - set(pd.get_dummies(df, columns=['tipo', 'cambio', 'cor', 'anunciante'], drop_first=True).columns)

{'anunciante_Acessórios e serviços para autos',
 'cambio_Automatizada',
 'cor_Branco',
 'tipo_Cupê'}

In [121]:
pipeline.fit(X, y)
pd.DataFrame([X.columns, pipeline.named_steps['regressor'].coef_]).T

,0,1
0,num_fotos,6201.452796
1,ano_de_fabricacao,11576.67032
2,ano_modelo,-4523.483629
3,hodometro,-23481.899578
4,num_portas,-4933.633473
5,blindado,4240.194366
6,vendedor_PJ,-8386.550846
7,entrega_delivery,-1515.255149
8,troca,1472.643735
9,elegivel_revisao,-0.0


In [156]:
df1 = pd.get_dummies(df, columns=['tipo', 'cambio', 'cor', 'anunciante'], drop_first=True)
df1['marca'] = df1['marca'].astype('category').cat.codes
df1['modelo'] = df1['modelo'].astype('category').cat.codes
df1['versao'] = df1['versao'].astype('category').cat.codes
df1['estado_vendedor'] = df1['estado_vendedor'].astype('category').cat.codes
df1.drop(columns='cidade_vendedor', inplace=True)


# from sklearn.compose import ColumnTransformer

X = df1.drop(columns=['preco', 'id'])
y = df1.preco

cat_cols = X.select_dtypes('object').columns.tolist()
# encoder = OrdinalEncoder()
# encoder.fit_transform(X)

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('categorical', OrdinalEncoder(), cat_cols)
#     ]
# )

kfold = KFold(n_splits=5, random_state=31, shuffle=True)

pipeline = Pipeline([
    # ('encoder', OneHotEncoder(categories='auto', drop='first')),
    # ('encoder', LabelEncoder()),
    # ('encoder', OrdinalEncoder()),
    # ('preprocessor', preprocessor),
    # ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=31))
    # ('regressor', LinearRegression())
])


scores = cross_val_score(pipeline, X, y, cv=kfold, scoring='r2')
print(scores)

[0.75459328 0.71582191 0.71679727 0.71852442 0.71470773]


In [181]:
df[~df.versao.str.slice(0,1).str.isdigit()]

,id,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,tipo,blindado,cor,vendedor_PJ,cidade_vendedor,estado_vendedor,anunciante,entrega_delivery,troca,elegivel_revisao,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado,preco
325,94089846439325118700825290312097791407,8,CHEVROLET,BOLT,EV 60 ELÉTRICO,2020,2021,24069.0,Automática,4,Utilitário esportivo,0,Cinza,0,Tupã,São Paulo (SP),Pessoa Física,0,0,0,0,1,1,1,1,1,1,0,255152.443907
396,264246424139552582303961695212337992215,17,NISSAN,LEAF,B12P 40 ELÉTRICO,2020,2020,28686.0,Automática,4,Picape,0,Preto,1,São Paulo,São Paulo (SP),Loja,0,0,0,1,1,1,1,1,1,0,0,270079.012211
469,170402357360433457134232326455564179831,8,NISSAN,LEAF,B12P 40 ELÉTRICO,2020,2020,36344.0,Automática,4,Picape,0,Branco,0,São Paulo,São Paulo (SP),Pessoa Física,0,0,0,1,1,0,1,1,1,0,0,159078.990809
837,185461663130960647149052633293291992667,14,NISSAN,LEAF,B12P 40 ELÉTRICO,2020,2021,8553.0,Automática,4,Picape,0,Cinza,1,Rio de Janeiro,Rio de Janeiro (RJ),Loja,0,0,0,1,0,0,1,1,0,0,0,223566.354929
1454,221079151358985685874726571251869762369,13,NISSAN,LEAF,B12P 40 ELÉTRICO,2020,2021,15727.0,Automática,4,Hatchback,0,Branco,1,São Paulo,São Paulo (SP),Loja,0,0,0,1,1,0,1,1,0,0,0,189930.782877
1969,41802501886300682558571456793009368840,15,VOLVO,XC40,P8 RECHARGE ELECTRIC BEV PURE AWD,2021,2022,15481.0,Automática,4,Utilitário esportivo,0,Cinza,1,Recife,Santa Catarina (SC),Concessionária,0,0,0,1,0,0,0,0,0,0,0,290341.398044
2490,126868065787629592054193790735367859138,8,RENAULT,ZOE,Z.E. 40 ELÉTRICO,2019,2019,25941.0,Automática,4,Hatchback,0,Branco,0,Belo Horizonte,Paraná (PR),Pessoa Física,0,0,0,1,0,1,0,0,0,0,0,109126.420820
3170,196042682123856742567037161235202591920,17,AUDI,E-TRON,ELÉTRICO SPORTBACK PERFORMANCE BLACK QUATTRO,2020,2021,26082.0,Automática,4,Utilitário esportivo,0,Cinza,1,Unaí,Minas Gerais (MG),Loja,0,0,0,1,1,0,0,0,0,0,0,227635.772921
3631,324488731727798327368807353277984338147,13,AUDI,E-TRON,ELÉTRICO SPORTBACK PERFORMANCE QUATTRO,2021,2021,23625.0,Automática,4,Utilitário esportivo,0,Cinza,1,Medianeira,Paraná (PR),Loja,0,0,0,1,0,0,0,0,0,0,0,296591.237987
3943,42310328341864571267391871022530846387,8,NISSAN,LEAF,B12P 40 ELÉTRICO,2020,2021,32234.0,Automática,4,Hatchback,0,Branco,0,Rio de Janeiro,Rio de Janeiro (RJ),Pessoa Física,0,0,0,1,1,0,1,1,1,0,0,192486.764953


In [199]:
df[~df['versao'].str.slice(0,3).str.match(r'^\d\.\d$')].sort_values('versao').versao.unique()



array(['4S ELÉTRICO', '90 KW EV400 SE AWD ELÉTRICO', 'B12P 40 ELÉTRICO',
       'BEV 8V ELÉTRICO 4P AUTOMÁTICO', 'ELÉTRICO',
       'ELÉTRICO PERFORMANCE QUATTRO',
       'ELÉTRICO SPORTBACK PERFORMANCE BLACK QUATTRO',
       'ELÉTRICO SPORTBACK PERFORMANCE QUATTRO',
       'ELÉTRICO eDRIVE BEV AUTOMÁTICO',
       'ELÉTRICO eDRIVE BEV FULL AUTOMÁTICO', 'EV 60 ELÉTRICO',
       'P8 RECHARGE ELECTRIC BEV PURE AWD', 'TURBO ELÉTRICO',
       'Z.E. 40 ELÉTRICO'], dtype=object)

In [195]:
df[df.versao.str.contains('ELETRI|ELÉTRI|ELECTRI')].sort_values('versao').versao.unique()

array(['4S ELÉTRICO', '90 KW EV400 SE AWD ELÉTRICO', 'B12P 40 ELÉTRICO',
       'BEV 8V ELÉTRICO 4P AUTOMÁTICO', 'ELÉTRICO',
       'ELÉTRICO PERFORMANCE QUATTRO',
       'ELÉTRICO SPORTBACK PERFORMANCE BLACK QUATTRO',
       'ELÉTRICO SPORTBACK PERFORMANCE QUATTRO',
       'ELÉTRICO eDRIVE BEV AUTOMÁTICO',
       'ELÉTRICO eDRIVE BEV FULL AUTOMÁTICO', 'EV 60 ELÉTRICO',
       'P8 RECHARGE ELECTRIC BEV PURE AWD', 'TURBO ELÉTRICO',
       'Z.E. 40 ELÉTRICO'], dtype=object)

In [190]:
df[df.versao.str.contains('ELET|ELÉT|ELEC')].sort_values('versao').versao.unique()

array(['2.0 ECOBOOST GASOLINA WILDTRAK 4X4 SELECTSHIFT',
       '2.5 LE 4X4 CD TURBO ELETRONIC DIESEL 4P MANUAL',
       '2.5 LE ATTACK 4X4 CD TURBO ELETRONIC DIESEL 4P AUTOMÁTICO',
       '2.5 S 4X2 CD TURBO ELETRONIC DIESEL 4P MANUAL',
       '2.5 SE ATTACK 4X2 CD TURBO ELETRONIC DIESEL 4P MANUAL',
       '2.5 SEL 4X4 CD TURBO ELETRONIC DIESEL 4P AUTOMÁTICO',
       '2.5 SL 4X4 CD TURBO ELETRONIC DIESEL 4P AUTOMÁTICO',
       '2.5 SV ATTACK 10 ANOS 4X4 CD TURBO ELETRONIC DIESEL 4P MANUAL',
       '2.5 SV ATTACK 4X2 CD TURBO ELETRONIC DIESEL 4P MANUAL',
       '2.5 SV ATTACK 4X4 CD TURBO ELETRONIC DIESEL 4P AUTOMÁTICO',
       '2.5 XE 4X2 CD TURBO ELETRONIC DIESEL 4P MANUAL',
       '2.5 XE 4X4 CD TURBO ELETRONIC DIESEL 4P MANUAL',
       '2.8 COLINA 4X2 CS 12V TURBO ELECTRONIC INTERCOOLER DIESEL 2P MANUAL',
       '2.8 COLINA 4X4 CD 12V TURBO ELECTRONIC INTERCOOLER DIESEL 4P MANUAL',
       '2.8 EXECUTIVE 4X4 CD 12V TURBO ELECTRONIC INTERCOOLER DIESEL 4P MANUAL',
       '2.8 TORNADO 

In [211]:
df[(df.versao.str.contains('HATCH'))]

,id,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,tipo,blindado,cor,vendedor_PJ,cidade_vendedor,estado_vendedor,anunciante,entrega_delivery,troca,elegivel_revisao,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado,preco
1701,170778200322433556409975352799349821559,8,BMW,130i,3.0 SPORT HATCH 24V GASOLINA 4P AUTOMÁTICO,1999,2011,144348.0,Automática,4,Hatchback,0,Preto,0,São Paulo,São Paulo (SP),Pessoa Física,0,0,0,1,0,0,1,1,0,0,0,58009.106770
1788,138629981022954720165123360612211459759,8,FORD,FIESTA,1.6 SE HATCH 16V FLEX 4P MANUAL,2016,2017,85590.0,Manual,4,Sedã,0,Branco,1,Canoinhas,Santa Catarina (SC),Loja,1,1,0,1,0,0,0,0,0,0,0,44947.737467
2595,304938004264200688414949546253993740636,8,FORD,FOCUS,2.0 TITANIUM HATCH 16V FLEX 4P AUTOMÁTICO,2013,2014,90876.0,Automática,4,Hatchback,0,Preto,0,Mogi das Cruzes,São Paulo (SP),Pessoa Física,0,0,0,0,0,1,1,1,0,1,0,52710.417011
2677,103364982067322159502650852050056310146,8,FORD,FIESTA,1.6 TITANIUM HATCH 16V FLEX 4P POWERSHIFT,2013,2013,84820.0,Automatizada,4,Hatchback,0,Preto,0,Rio de Janeiro,São Paulo (SP),Pessoa Física,0,0,0,0,0,1,1,1,0,0,0,57842.345509
2999,143633099461413863698934917863030205925,8,FORD,FIESTA,1.6 MPI CLASS HATCH 8V FLEX 4P MANUAL,2000,2012,162345.0,Manual,4,Hatchback,0,Preto,0,Rio de Janeiro,São Paulo (SP),Pessoa Física,0,0,0,0,0,0,1,1,0,0,0,29741.010385
3954,268908174483047025882708895452133201482,8,FORD,FOCUS,2.0 TITANIUM HATCH 16V FLEX 4P AUTOMÁTICO,2014,2014,67759.0,Automática,4,Hatchback,0,Preto,0,Goiânia,Minas Gerais (MG),Pessoa Física,0,0,0,1,0,1,1,0,0,1,0,77779.370289
4635,312944905666610407536855581399808839382,8,FORD,FIESTA,1.6 SEL HATCH 16V FLEX 4P MANUAL,2016,2017,100356.0,Manual,4,Hatchback,0,Branco,0,Medianeira,Paraná (PR),Pessoa Física,0,0,0,0,0,0,1,0,0,0,0,53047.188524
5307,114795814491135626770673569326780363988,8,FORD,FIESTA,1.5 S HATCH 16V FLEX 4P MANUAL,2013,2014,88161.0,Manual,4,Picape,0,Branco,0,Recife,Minas Gerais (MG),Pessoa Física,0,0,0,1,0,0,0,0,0,0,0,46190.882161
5879,167401423052323247661642540367279879058,8,FORD,FIESTA,1.6 SEL HATCH 16V FLEX 4P MANUAL,2017,2017,94610.0,Manual,4,Hatchback,0,Branco,0,Tianguá,Rio Grande do Sul (RS),Pessoa Física,0,0,0,1,0,0,0,0,0,0,0,44390.215346
6542,274356436005731229133533395400518267356,8,FORD,FIESTA,1.6 SEL HATCH 16V FLEX 4P MANUAL,2016,2017,100734.0,Manual,4,Hatchback,0,Branco,0,Medianeira,Paraná (PR),Pessoa Física,0,0,0,0,0,0,0,0,0,0,0,56608.623016


In [138]:
X.select_dtypes('object').columns.tolist()

['marca',
 'modelo',
 'versao',
 'cambio',
 'tipo',
 'cor',
 'cidade_vendedor',
 'estado_vendedor',
 'anunciante']